# ANSWER KEY: Capstone Project Solution

This notebook contains complete solutions for the StreamCart Retention System capstone.

---

In [ ]:
# Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

try:
    import lightgbm as lgb
except:
    !pip install lightgbm -q
    import lightgbm as lgb

# Load data
DATA_URL = "https://raw.githubusercontent.com/189investmentai/ml-foundations-interactive/main/"
customers = pd.read_csv(DATA_URL + "streamcart_customers.csv")
print(f"Loaded {len(customers):,} customers")

---

# Part 1: Problem Framing - SOLUTION

In [ ]:
# SOLUTION: Completed framing template

framing_solution = """
=== ML PROBLEM FRAMING ===

1. Business Goal: Reduce customer churn by targeting at-risk customers with retention calls.

2. ML Task Type: Binary Classification

3. Target (y): churn_30d - whether a customer cancels within 30 days of the snapshot date.

4. Prediction Point: Weekly, on the snapshot_date for each customer.

5. Features (X): Historical data available before snapshot_date:
   - Account info: tenure_months, subscription_plan
   - Activity: logins_last_30d, orders_last_30d
   - Support: support_tickets_last_30d
   - Sentiment: nps_score
   - Derived: orders_per_month, support_intensity, engagement_score

6. Success Metric: Precision@500 (accuracy in the top 500 predictions)
   - Why: We have a capacity constraint of 500 calls/week
   - Secondary: Lift over random baseline

7. Business Action: Top 500 predicted churners receive retention calls with discount offers.
   - If prediction = high risk AND customer is called → 30% chance of saving them
   - Each saved customer = $200 value
   - Each call = $15 cost
"""
print(framing_solution)

In [ ]:
# SOLUTION: Baseline calculation

base_churn_rate = customers['churn_30d'].mean()
calls_per_week = 500
save_rate = 0.30
value_per_save = 200
cost_per_call = 15

# Random selection baseline
expected_churners_random = calls_per_week * base_churn_rate
expected_saves_random = expected_churners_random * save_rate
net_value_random = (expected_saves_random * value_per_save) - (calls_per_week * cost_per_call)

print(f"=== RANDOM SELECTION BASELINE ===")
print(f"Churn rate: {base_churn_rate:.1%}")
print(f"Expected churners in 500 random calls: {expected_churners_random:.0f}")
print(f"Expected saves: {expected_saves_random:.0f}")
print(f"Net value: ${net_value_random:,.0f}")

---

# Part 2: Feature Engineering - SOLUTION

In [ ]:
# SOLUTION: Feature engineering

df = customers.copy()

# Feature 1: Orders per month (ratio - normalizes by tenure)
df['orders_per_month'] = df['orders_last_30d'] / (df['tenure_months'] + 1)

# Feature 2: Support intensity (high tickets relative to tenure = problem)
df['support_intensity'] = df['support_tickets_last_30d'] / (df['tenure_months'] + 1)

# Feature 3: Engagement score (logins per order - high ratio might mean browsing without buying)
df['engagement_score'] = df['logins_last_30d'] / (df['orders_last_30d'] + 1)

# Feature 4: Low NPS flag (detractors)
df['is_detractor'] = (df['nps_score'] <= 6).astype(int)

# Feature 5: New customer flag (first 3 months are risky)
df['is_new_customer'] = (df['tenure_months'] <= 3).astype(int)

print("Engineered features:")
print(df[['orders_per_month', 'support_intensity', 'engagement_score', 'is_detractor', 'is_new_customer']].describe())

In [ ]:
# SOLUTION: Leakage audit

leakage_audit_solution = """
=== FEATURE LEAKAGE AUDIT ===

| Feature | Source | Available at prediction? | Safe? |
|---------|--------|-------------------------|-------|
| tenure_months | Account creation date | Yes - historical | ✓ |
| logins_last_30d | Activity logs | Yes - historical | ✓ |
| orders_last_30d | Order history | Yes - historical | ✓ |
| support_tickets_last_30d | Support system | Yes - historical | ✓ |
| nps_score | Survey response | Yes - collected before snapshot | ✓ |
| orders_per_month | orders / tenure | Yes - derived from safe features | ✓ |
| support_intensity | tickets / tenure | Yes - derived from safe features | ✓ |
| engagement_score | logins / orders | Yes - derived from safe features | ✓ |
| is_detractor | nps_score <= 6 | Yes - derived from safe feature | ✓ |
| is_new_customer | tenure <= 3 | Yes - derived from safe feature | ✓ |

EXCLUDED (leaky):
- churn_date: Only known AFTER churn happens
- cancel_reason: Only known AFTER cancellation

Confirm: ✓ All features use data available at prediction time.
"""
print(leakage_audit_solution)

In [ ]:
# SOLUTION: Final feature set

all_features = [
    # Base features
    'tenure_months',
    'logins_last_30d',
    'orders_last_30d',
    'support_tickets_last_30d',
    'nps_score',
    # Engineered features
    'orders_per_month',
    'support_intensity',
    'engagement_score',
    'is_detractor',
    'is_new_customer'
]

X = df[all_features].fillna(0)
y = df['churn_30d']

print(f"Features: {len(all_features)}")
print(f"Samples: {len(X):,}")

---

# Part 3: Model Training - SOLUTION

In [ ]:
# SOLUTION: Train/val/test split

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp)

print(f"Train: {len(X_train):,} | Val: {len(X_val):,} | Test: {len(X_test):,}")

In [ ]:
# SOLUTION: Logistic Regression

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

model_lr = LogisticRegression(random_state=42, max_iter=1000, C=0.5)
model_lr.fit(X_train_scaled, y_train)

probs_lr_val = model_lr.predict_proba(X_val_scaled)[:, 1]
auc_lr_val = roc_auc_score(y_val, probs_lr_val)
print(f"Logistic Regression Val AUC: {auc_lr_val:.3f}")

In [ ]:
# SOLUTION: LightGBM with early stopping

model_lgb = lgb.LGBMClassifier(
    n_estimators=500,
    num_leaves=31,
    max_depth=6,
    min_child_samples=20,
    learning_rate=0.05,
    reg_alpha=0.1,
    reg_lambda=0.1,
    random_state=42,
    verbose=-1
)

model_lgb.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)]
)

probs_lgb_val = model_lgb.predict_proba(X_val)[:, 1]
auc_lgb_val = roc_auc_score(y_val, probs_lgb_val)
print(f"LightGBM Val AUC: {auc_lgb_val:.3f}")
print(f"Trees used: {model_lgb.best_iteration_}")

---

# Part 4: Evaluation - SOLUTION

In [ ]:
# SOLUTION: Test set evaluation

probs_lr_test = model_lr.predict_proba(X_test_scaled)[:, 1]
probs_lgb_test = model_lgb.predict_proba(X_test)[:, 1]

auc_lr_test = roc_auc_score(y_test, probs_lr_test)
auc_lgb_test = roc_auc_score(y_test, probs_lgb_test)

print(f"Test AUC - Logistic Regression: {auc_lr_test:.3f}")
print(f"Test AUC - LightGBM: {auc_lgb_test:.3f}")

In [ ]:
# SOLUTION: Precision@K

def precision_at_k(y_true, y_proba, k):
    top_k_idx = np.argsort(y_proba)[::-1][:k]
    return y_true.iloc[top_k_idx].mean()

K = 500
k_test = int(K * len(y_test) / len(y))

precision_random = y_test.mean()
precision_lr = precision_at_k(y_test, probs_lr_test, k_test)
precision_lgb = precision_at_k(y_test, probs_lgb_test, k_test)

lift_lr = precision_lr / precision_random
lift_lgb = precision_lgb / precision_random

print(f"\nPrecision@{k_test}:")
print(f"  Random: {precision_random:.1%}")
print(f"  Logistic Regression: {precision_lr:.1%} (Lift: {lift_lr:.1f}x)")
print(f"  LightGBM: {precision_lgb:.1%} (Lift: {lift_lgb:.1f}x)")

In [ ]:
# SOLUTION: Business impact

best_precision = max(precision_lr, precision_lgb)
best_model = "LightGBM" if precision_lgb > precision_lr else "Logistic Regression"

expected_churners_model = 500 * best_precision
expected_saves_model = expected_churners_model * save_rate
net_value_model = (expected_saves_model * value_per_save) - (500 * cost_per_call)
value_improvement = net_value_model - net_value_random

print(f"\n=== BUSINESS IMPACT ({best_model}) ===")
print(f"Expected churners in top 500: {expected_churners_model:.0f} (vs {expected_churners_random:.0f} random)")
print(f"Expected saves: {expected_saves_model:.0f} (vs {expected_saves_random:.0f} random)")
print(f"Net value per week: ${net_value_model:,.0f}")
print(f"\nWeekly improvement: ${value_improvement:,.0f}")
print(f"Annual improvement: ${value_improvement * 52:,.0f}")

---

# Part 5: Communication - SOLUTION

In [ ]:
# SOLUTION: PM Update

pm_update_solution = f"""
=== WEEKLY UPDATE: CHURN PREDICTION MODEL ===

Hi Sarah,

WHAT WE BUILT
We built a machine learning model that predicts which customers are likely to cancel 
in the next 30 days. The model uses customer behavior data (logins, orders, support 
tickets) to identify at-risk customers before they churn.

KEY RESULTS
Using our model to select which 500 customers to call each week:
- We expect to reach {expected_churners_model:.0f} actual churners vs {expected_churners_random:.0f} with random selection
- That's {lift_lgb:.1f}x more churners than current random approach
- Projected annual value: ${value_improvement * 52:,.0f} in additional retained customers

RECOMMENDATION
I recommend we deploy the LightGBM model. While slightly less interpretable than 
simpler alternatives, it delivers the best precision where it matters (top 500). 
The model correctly identifies customers with high support tickets and low engagement 
as highest risk.

NEXT STEPS
1. Pilot: Run model predictions alongside current process for 2 weeks to validate
2. Integration: Connect model output to the retention team's call queue system
3. Monitoring: Set up weekly dashboards tracking precision and save rate

Happy to walk through the details whenever works for you.

Best,
[Your Name]
"""

print(pm_update_solution)
print(f"\nWord count: {len(pm_update_solution.split())} words")

---

# Grading Notes

## Part 1 (20 points)
- Full credit: All 7 lines completed with specific, correct answers
- Partial credit: Missing time windows or vague success metrics

## Part 2 (20 points)
- Full credit: 3+ meaningful features, complete leakage audit, no future data
- Deductions: Leaky features, features that don't make business sense

## Part 3 (20 points)
- Full credit: Two models trained, early stopping used, overfitting checked
- Deductions: No validation set, no early stopping, large train/test gap

## Part 4 (20 points)
- Full credit: Precision@K reported, Lift calculated, $ impact computed
- Deductions: Wrong metrics, no comparison to baseline

## Part 5 (20 points)
- Full credit: Clear update, business metrics, recommendation, next steps
- Deductions: Too technical, no concrete numbers, missing recommendation